In [5]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.CepsMy1Experiment import CepsMy1Experiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [6]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [7]:
train_instances[0].filepath

WindowsPath('C:/Users/zakrz/Documents/DataScience/praca magisterska/raw-algorithm-portfolios/data/TSP/CEPS_benchmark/cluster_netgen/40.tsp')

In [8]:
train_instances[0].mutate()

Concorde C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios\data\TSP\CEPS_generated\0.tsp


(<src.instance.TSP_Instance.TSP_Instance at 0x1e481ee3c40>, 100.0)

In [3]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
t_c = 200
K = 2
n = 2
t_ini = train_instances.size * 10 * 2
t_i = 10
max_iter = 2

experiment = CepsExperiment(
    t_c=t_c,
    t_ini=t_ini,
    t_i=t_i,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-11 21:37:19] INFO      [CEPS] Start!
[2024-12-11 21:37:19] INFO      Initialization
[2024-12-11 21:37:19] INFO      Solver 1/2
[2024-12-11 21:37:19] INFO      Random config 1/2
[2024-12-11 21:37:19] DEBUG     =================================  Portfolio  ==================================
[2024-12-11 21:37:19] DEBUG     2220588739511820280 --> ASCENT_CANDIDATES=52;BACKBONE_TRIALS=1;BACKTRACKING=YES;CANDIDATE_SET_TYPE=ALPHA;EXTRA_CANDIDATES=7;EXTRA_CANDIDATE_SET_TYPE=QUADRANT;GAIN23=NO;GAIN_CRITERION=YES;INITIAL_STEP_SIZE=1;INITIAL_TOUR_ALGORITHM=QUICK-BORUVKA;INITIAL_TOUR_FRACTION=0.5820197920751;KICKS=3;KICK_TYPE=0;MAX_BREADTH=954863081;MAX_CANDIDATES=10;MOVE_TYPE=3;PATCHING_A=3;PATCHING_C=1;POPULATION_SIZE=8;RESTRICTED_SEARCH=YES;SUBGRADIENT=YES;SUBSEQUENT_MOVE_TYPE=6;SUBSEQUENT_PATCHING=NO
[2024-12-11 21:37:19] DEBUG     ================================================================================
[2024-12-11 21:37:19] DEBUG     executor start
[2024-12-11 21:37:19] DEBUG

In [3]:
t_c = 60
K = 2
n = 2
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = GlobalExperiment(
    t_c=t_c,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

[2024-12-10 20:56:48] INFO      [GLOBAL] Start!


In [3]:
t_c = 60
K = 2
n = 2
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = GlobalExperiment(
    t_c=t_c,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-09 23:17:26] INFO      [GLOBAL] Start!
[2024-12-09 23:17:26] INFO      Attempt 1/2
[2024-12-09 23:17:26] DEBUG     SMAC configuration, time: [60. 60.]
[2024-12-09 23:17:26] DEBUG     SMAC iteration 1, configuration: {'0__ASCENT_CANDIDATES': 45, '0__BACKBONE_TRIALS': 0, '0__BACKTRACKING': 'YES', '0__CANDIDATE_SET_TYPE': 'QUADRANT', '0__EXTRA_CANDIDATES': 0, '0__EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT', '0__GAIN23': 'NO', '0__GAIN_CRITERION': 'NO', '0__INITIAL_STEP_SIZE': 1, '0__INITIAL_TOUR_ALGORITHM': 'NEAREST-NEIGHBOR', '0__INITIAL_TOUR_FRACTION': 0.1410864457464, '0__KICKS': 3, '0__KICK_TYPE': 5, '0__MAX_BREADTH': 1850229859, '0__MAX_CANDIDATES': 4, '0__MOVE_TYPE': 3, '0__PATCHING_A': 4, '0__PATCHING_C': 2, '0__POPULATION_SIZE': 8, '0__RESTRICTED_SEARCH': 'YES', '0__SUBGRADIENT': 'YES', '0__SUBSEQUENT_MOVE_TYPE': 5, '0__SUBSEQUENT_PATCHING': 'NO', '1__ASCENT_CANDIDATES': 51, '1__BACKBONE_TRIALS': 0, '1__BACKTRACKING': 'YES', '1__CANDIDATE_SET_TYPE': 'DELAUNAY', '1__EXTRA_CANDID

In [3]:
t_c = 60
K = 2
n = 2
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = ParhydraExperiment(
    t_c=t_c,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-09 23:20:45] INFO      [PARHYDRA] Start!
[2024-12-09 23:20:45] INFO      Solver 1/2
[2024-12-09 23:20:45] INFO      Attempt 1/2
[2024-12-09 23:20:45] DEBUG     SMAC configuration, time: [60.]
[2024-12-09 23:20:45] DEBUG     SMAC iteration 1, configuration: {'0__ASCENT_CANDIDATES': 52, '0__BACKBONE_TRIALS': 1, '0__BACKTRACKING': 'YES', '0__CANDIDATE_SET_TYPE': 'ALPHA', '0__EXTRA_CANDIDATES': 7, '0__EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT', '0__GAIN23': 'NO', '0__GAIN_CRITERION': 'YES', '0__INITIAL_STEP_SIZE': 1, '0__INITIAL_TOUR_ALGORITHM': 'QUICK-BORUVKA', '0__INITIAL_TOUR_FRACTION': 0.5820197920751, '0__KICKS': 3, '0__KICK_TYPE': 0, '0__MAX_BREADTH': 954863081, '0__MAX_CANDIDATES': 10, '0__MOVE_TYPE': 3, '0__PATCHING_A': 3, '0__PATCHING_C': 1, '0__POPULATION_SIZE': 8, '0__RESTRICTED_SEARCH': 'YES', '0__SUBGRADIENT': 'YES', '0__SUBSEQUENT_MOVE_TYPE': 6, '0__SUBSEQUENT_PATCHING': 'NO'}
[2024-12-09 23:20:45] DEBUG     executor start
[2024-12-09 23:20:45] DEBUG     (0, 0) fn submitt

In [ ]:
# train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
#     filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
#     train_size=5,
#     seed=0,
# )

# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = sum([i * train_instances.size * 10 for i in range(1, K + 1)]) * 2
# t_i = 10
# max_iter = 1

# experiment = CepsMy1Experiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [22]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [8]:
from src.database import db_connect
conn = db_connect()

In [23]:
import sqlite3
# conn = sqlite3.connect("_archive/2/GLOBAL.db")
conn = sqlite3.connect("database/2024_12_11_21_49_39.db")

In [24]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2220588739511820280,52,1,YES,ALPHA,7,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.582020,3,0,954863081,10,3,3,1,8,YES,YES,6,NO
1,2051815471081033189,45,0,YES,DELAUNAY,10,QUADRANT,YES,YES,3,SIERPINSKI,0.842342,3,5,1010150627,9,6,3,1,26,YES,NO,4,NO
2,574052600841816340,58,1,YES,QUADRANT,6,QUADRANT,NO,YES,5,GREEDY,0.569101,2,5,737143853,6,2,3,4,37,NO,NO,6,YES
3,1333412370196243425,58,1,YES,QUADRANT,6,QUADRANT,YES,YES,1,BORUVKA,0.788739,3,4,1321047180,5,4,4,2,28,YES,NO,0,NO
4,607108495934423959,42,0,YES,DELAUNAY,3,QUADRANT,YES,YES,2,GREEDY,0.972919,4,0,143733986,2,4,1,4,53,NO,YES,5,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,545033399970933276,53,0,NO,DELAUNAY,1,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.599166,0,0,1630604567,6,5,1,1,57,YES,NO,2,NO
513,2248611603799359049,48,1,NO,ALPHA,5,QUADRANT,YES,NO,1,NEAREST-NEIGHBOR,0.691602,1,4,241512438,3,4,5,0,58,NO,NO,6,NO
514,101670489796509015,53,1,NO,DELAUNAY,6,QUADRANT,YES,YES,5,SIERPINSKI,0.721767,5,4,725673235,2,6,0,5,2,YES,NO,3,NO
515,747531208815134112,50,0,YES,QUADRANT,10,QUADRANT,YES,NO,4,NEAREST-NEIGHBOR,0.198489,5,4,1665278032,7,3,1,3,87,YES,YES,6,NO


In [34]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
663,data/TSP/CEPS_benchmark/implosion/23.tsp
664,data/TSP/CEPS_benchmark/expansion/42.tsp
665,data/TSP/CEPS_benchmark/compression/17.tsp
666,data/TSP/CEPS_benchmark/cluster/47.tsp


In [37]:
df_instances.loc[df_instances["id"].str.contains("CEPS_generated")]

,id
30,data/TSP/CEPS_generated/0.tsp
31,data/TSP/CEPS_generated/1.tsp
32,data/TSP/CEPS_generated/2.tsp
33,data/TSP/CEPS_generated/3.tsp
34,data/TSP/CEPS_generated/4.tsp
...,...
193,data/TSP/CEPS_generated/294.tsp
194,data/TSP/CEPS_generated/295.tsp
195,data/TSP/CEPS_generated/297.tsp
196,data/TSP/CEPS_generated/300.tsp


In [38]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-11 20:49:51
1,data/TSP/CEPS_benchmark/grid/04.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-11 20:49:51
2,data/TSP/CEPS_benchmark/grid/33.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-11 20:49:51
3,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-11 20:49:51
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-11 20:49:51
...,...,...,...,...,...,...
31909,data/TSP/CEPS_benchmark/cluster/47.tsp,674619091039080205,7.62,7.62,test3,2024-12-12 11:53:22
31910,data/TSP/CEPS_benchmark/expansion/22.tsp,2152573642869061282,0.72,0.72,test3,2024-12-12 11:53:22
31911,data/TSP/CEPS_benchmark/expansion/22.tsp,1225198965801379609,5.12,5.12,test3,2024-12-12 11:53:22
31912,data/TSP/CEPS_benchmark/expansion/22.tsp,2150300497401555222,3.36,3.36,test3,2024-12-12 11:53:22


In [28]:
df.sort_values(by=["solver_id", "instance_id"])

,instance_id,solver_id,cost,time,comment,created_at
5285,data/TSP/CEPS_benchmark/cluster/15.tsp,1000090002772723721,100.0,10.0,configuration,2024-12-11 23:07:23
5306,data/TSP/CEPS_benchmark/cluster/21.tsp,1000090002772723721,100.0,10.0,configuration,2024-12-11 23:08:31
5299,data/TSP/CEPS_benchmark/cluster/37.tsp,1000090002772723721,100.0,10.0,configuration,2024-12-11 23:07:58
5304,data/TSP/CEPS_benchmark/cluster/45.tsp,1000090002772723721,100.0,10.0,configuration,2024-12-11 23:08:29
5280,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,1000090002772723721,100.0,10.0,configuration,2024-12-11 23:06:46
...,...,...,...,...,...,...
16803,data/TSP/CEPS_generated/92.tsp,997343237269373437,100.0,10.0,configuration,2024-12-12 06:00:59
16804,data/TSP/CEPS_generated/93.tsp,997343237269373437,100.0,10.0,configuration,2024-12-12 06:01:00
16805,data/TSP/CEPS_generated/94.tsp,997343237269373437,100.0,10.0,configuration,2024-12-12 06:01:06
16806,data/TSP/CEPS_generated/95.tsp,997343237269373437,100.0,10.0,configuration,2024-12-12 06:01:15


In [54]:
df["comment"].value_counts()

configuration     22659
initialization     2880
test1              1880
test2              1880
test3              1880
mutation            660
validation           75
Name: comment, dtype: int64

In [30]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
2880,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,928318422486750652,100.0,10.0,configuration,2024-12-11 22:06:50
2881,data/TSP/CEPS_benchmark/grid/04.tsp,928318422486750652,100.0,10.0,configuration,2024-12-11 22:06:50
2882,data/TSP/CEPS_benchmark/grid/33.tsp,928318422486750652,100.0,10.0,configuration,2024-12-11 22:06:53
2883,data/TSP/CEPS_benchmark/rotation/45.tsp,928318422486750652,100.0,10.0,configuration,2024-12-11 22:06:53
2884,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,928318422486750652,100.0,10.0,configuration,2024-12-11 22:06:53
...,...,...,...,...,...,...
26269,data/TSP/CEPS_generated/83.tsp,152376327931693906,100.0,10.0,configuration,2024-12-12 10:58:45
26270,data/TSP/CEPS_generated/84.tsp,152376327931693906,100.0,10.0,configuration,2024-12-12 10:58:46
26271,data/TSP/CEPS_generated/85.tsp,152376327931693906,100.0,10.0,configuration,2024-12-12 10:58:46
26272,data/TSP/CEPS_generated/86.tsp,152376327931693906,100.0,10.0,configuration,2024-12-12 10:58:46


In [31]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,1225198965801379609,2150300497401555222,2152573642869061282,674619091039080205,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,100.00,1.31,6.10,5.39,1,1.31
data/TSP/CEPS_benchmark/cluster/01.tsp,7.41,1.03,1.31,100.00,1,1.03
data/TSP/CEPS_benchmark/cluster/02.tsp,1.07,100.00,3.84,5.98,0,1.07
data/TSP/CEPS_benchmark/cluster/03.tsp,0.52,100.00,3.51,3.48,0,0.52
data/TSP/CEPS_benchmark/cluster/04.tsp,100.00,100.00,100.00,100.00,0,100.00
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,1.44,1.80,3.68,0.46,3,0.46
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,1.13,6.14,0.49,4.58,2,0.49
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,0.64,100.00,5.06,0.40,3,0.40


In [32]:
_["min"].describe()

count    470.000000
mean       5.031362
std       17.928458
min        0.080000
25%        0.570000
50%        1.060000
75%        2.337500
max      100.000000
Name: min, dtype: float64

In [33]:
(_["min"] > 90).value_counts()

False    454
True      16
Name: min, dtype: int64

In [38]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

1    277
0    105
2      6
Name: best, dtype: int64

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index